In [56]:
award_names = ['best Motion Picture – Drama', 'Best Director – Motion Picture', 'Best Motion Picture – Musical or Comedy', 'Best Television Series – Drama', 'Best Actor in a Motion Picture – Drama']
new_award_names = [0]*len(award_names)
for i in range(len(award_names)):
    new_award_names[i] = award_names[i].replace(" in ", " ").replace(" a ", " ").replace(" or ", " ").replace(" – ", " ").replace(" - ", " ").lower().split()
new_award_names

[['best', 'motion', 'picture', 'drama'],
 ['best', 'director', 'motion', 'picture'],
 ['best', 'motion', 'picture', 'musical', 'comedy'],
 ['best', 'television', 'series', 'drama'],
 ['best', 'actor', 'motion', 'picture', 'drama']]

In [57]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from collections import defaultdict

In [36]:
df = pd.read_json('../gg2013.json')
df = df['text']
df = df[~df.str.contains('think|thinking|should|maybe', case = False)]
df = df.str.replace('#GoldenGlobes|golden|globes|globe', "", case = False)
wins_df = df[df.str.contains('won best|wins best|goes to', case = False)]

wins_df

3490      RT @yeahitsaline_: And the   to "Best Twitter ...
4511      RT @EricaDeLong:  #EricaRedCarpet Dustin Hoffm...
7462      And  goes tooooooooo... Oğlum kara kışta böyle...
9194      Biggest zinger so far at the  goes to Amy Poeh...
9730      Watching the  ! I say silver linings wins best...
                                ...                        
174473    RT @playbill: "Les Miserables" wins Best Motio...
174567    RT @nerdywonka: Ben Affleck won Best Director ...
174576    Best Actor, Drama:   For Film Goes To Daniel D...
174578    Best Actor, Drama:   For Film Goes To Daniel D...
174580    Best Actor, Drama:   For Film Goes To Daniel D...
Name: text, Length: 3806, dtype: object

In [37]:
# #for tweet in wins_df:
# tweet = 'My name is the best name musical ever and ever 1 2 3 actor director forever no one has a better name because 1 2 3 mine is the best'
# split = tweet.lower().split()
# index = split.index("best")
# len_after = len(split) - index
# if len_after > 12:
#     award_phrase = split[index:index+12]
# else:
#     award_phrase = split[index:]

# print(award_phrase)

# name_count = [0]*len(new_award_names)
# for i in range(len(new_award_names)):
#     match_length = len(set(new_award_names[i]).intersection(award_phrase))
#     name_count[i] = match_length

# max_val = max(name_count)

# if max_val > 1:
#     ## get the person from the phrase that came right before "won"
#     for i in range(name_count):
#         if name_count[i] == max_val:
#             ## tally that person's name in the awards -> people array
    

SyntaxError: unexpected EOF while parsing (<ipython-input-37-4650c27f912e>, line 29)

In [4]:
import numpy as np
import pandas as pd
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
from collections import defaultdict
nlp = spacy.load('en_core_web_md')
from string import punctuation

In [55]:
pattern1 = [{"ENT_TYPE": "PERSON", 'OP':'+'}, {"LEMMA": "win"}, {"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]
pattern2 = [{"ORTH":'Best'}, {"DEP": 'compound', 'OP':'*'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}, {"TEXT": "goes to"}, {"ENT_TYPE": "PERSON", 'OP':'+'}]

In [56]:
matcher = Matcher(nlp.vocab)
matcher.add("Winner", [pattern1, pattern2])

In [57]:
winners = {}
winners = defaultdict(lambda: 1, winners)
for i, text in wins_df.iteritems():
    doc = nlp(text)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        winners[str(span).strip(punctuation).strip()] = winners[str(span).strip(punctuation).strip()] +1

In [58]:
winners = sorted(winners.items(), key=lambda item: item[1], reverse = True)
print(winners)

[('Adele wins Best Original Song', 29), ('Christoph Waltz Wins Best Supporting Actor', 24), ('Christoph Waltz wins Best Supporting Actor', 13), ('Daniel Day-Lewis wins Best Actor Motion Picture - Drama', 13), ('Adele won Best Original Song', 6), ('Christoph Waltz won Best Supporting Actor', 5), ('Ben Affleck wins Best Director - Motion Picture', 5), ('Les Miserables wins Best Motion Picture', 5), ('Adele wins Best Original Song Motion Picture', 4), ('Jennifer Lawrence wins Best Actress Motion Picture Comedy', 4), ('Adele Wins Best Original Song', 4), ('Damian Lewis wins Best Actor TV Series', 3), ('Don Cheadle Wins Best TV Actor - Comedy', 3), ('Mychael Danna wins Best Original Score', 3), ('Ben Affleck wins Best Director prize', 3), ('Christoph Waltz wins Best Supporting Actor - Motion Picture', 2), ('Christoph Waltz WINS Best Supporting Actor', 2), ('Yayyyy Adele wins Best Original Song', 2), ('Christoph Waltz Wins Best Supportin Actor', 2), ('Glad Adele won Best Original Song', 2), 